In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

from datetime import datetime

In [2]:
seed = 42
cuda = True

batch_size = 64
log_interval = 1000

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True)

0it [00:00, ?it/s]

100%|█████████▉| 9887744/9912422 [00:26<00:00, 362420.07it/s]

Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz



0it [00:00, ?it/s]


 57%|█████▋    | 16384/28881 [00:00<00:00, 153524.65it/s]
32768it [00:00, 87489.86it/s]                            
0it [00:00, ?it/s]

Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz



 99%|█████████▉| 1630208/1648877 [00:04<00:00, 393106.07it/s]

0it [00:00, ?it/s]

Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz




  0%|          | 0/4542 [00:00<?, ?it/s]

8192it [00:00, 35086.52it/s]            

Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!


In [3]:
class Net1(nn.Module):
    def __init__(self):
        super(self.__class__, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)

In [4]:
class Lenet(nn.Module):
    def __init__(self):
        super(self.__class__, self).__init__()
        self.conv2_n = 50
        self.hiddens = 50*16
        
        self.conv1 = nn.Conv2d(1, 20, kernel_size=5)
        self.conv2 = nn.Conv2d(20, self.conv2_n, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(self.hiddens, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = F.tanh(F.max_pool2d(self.conv1(x), 2))
        x = F.tanh(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, self.hiddens)
        x = F.tanh(self.fc1(x))
        x = F.dropout(x, training=self.training, p=0.7)
        x = self.fc2(x)
        return F.log_softmax(x)

In [5]:
model = Lenet()
if cuda:
    model.cuda()

epochs = 40
lr = 0.01
momentum = 0.9

optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

9920512it [00:40, 362420.07it/s]                             

In [12]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data.item()))
            
def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        if cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += F.nll_loss(output, target, size_average=False).item() # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [13]:
%%time

start = datetime.now()
new = start

for epoch in range(1, epochs + 1):
    train(epoch)
    test()
    old = new
    new = datetime.now()
    print(">> %s - %s" % (str(new-old), str(new-start)))

/home/yannick/bin/anaconda3_2018.12/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Train Epoch: 1 [0/60000 (0%)]	Loss: 0.256842


/home/yannick/bin/anaconda3_2018.12/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:24: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


Test set: Average loss: 0.0623, Accuracy: 9815/10000 (98%)
>> 0:00:11.949046 - 0:00:11.949046
Train Epoch: 2 [0/60000 (0%)]	Loss: 0.090392
Test set: Average loss: 0.0490, Accuracy: 9851/10000 (98%)
>> 0:00:11.916971 - 0:00:23.866017
Train Epoch: 3 [0/60000 (0%)]	Loss: 0.055971
Test set: Average loss: 0.0463, Accuracy: 9863/10000 (98%)
>> 0:00:12.105589 - 0:00:35.971606
Train Epoch: 4 [0/60000 (0%)]	Loss: 0.078448
Test set: Average loss: 0.0450, Accuracy: 9861/10000 (98%)
>> 0:00:11.966944 - 0:00:47.938550
Train Epoch: 5 [0/60000 (0%)]	Loss: 0.063577
Test set: Average loss: 0.0437, Accuracy: 9878/10000 (98%)
>> 0:00:11.859318 - 0:00:59.797868
Train Epoch: 6 [0/60000 (0%)]	Loss: 0.037095
Test set: Average loss: 0.0380, Accuracy: 9882/10000 (98%)
>> 0:00:11.963834 - 0:01:11.761702
Train Epoch: 7 [0/60000 (0%)]	Loss: 0.069324
Test set: Average loss: 0.0420, Accuracy: 9872/10000 (98%)
>> 0:00:11.800502 - 0:01:23.562204
Train Epoch: 8 [0/60000 (0%)]	Loss: 0.174236
Test set: Average loss: 0.0

In [14]:
model

Lenet(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (conv2_drop): Dropout2d(p=0.5)
  (fc1): Linear(in_features=800, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=10, bias=True)
)